In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#import matplotlib.pyplot as plt
import psycopg2

In [2]:
fact_table_cols = [('Individual_Key', 'Resolved', 'Fatal', 'Special_Measure_Key', 'PHU_Key', 'Weather_Key', 'Onset_Date_Key', 'Reported_Date_Key', 'Test_Date_Key', 'Specimen_Date_Key')]
individual_cols = [('surrogate_key', 'individual_id', 'date', 'age_group', 'gender', 'cause_of_infection', 'outbreak_related', 'reporting_phu_city')]
fact_table = []
individual_table = []

def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        return psycopg2.connect(
            host="www.eecs.uottawa.ca",
            database="group_21",
            user="togun062",
            password="Ad6jyqyv8p",
            port="15432"
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


connection = connect()
print('***Connection established***')

cur = connection.cursor()
        
# execute a statement
cur.execute('SELECT * FROM fact_table_v2')
rows = cur.fetchall()
fact_table = rows
print(fact_table[:10])


print("\n\n")


cur.execute('SELECT * FROM individuals')
rows = cur.fetchall()
individual_table = rows
print(individual_table[:10])

cur.close()
connection.close()
print('***Connection terminated***')

Connecting to the PostgreSQL database...
***Connection established***
[(250, 'yes', 'no', 11, 3895, 123, 27, 32, 32, 29), (322, 'yes', 'no', 11, 3895, 110, 16, 19, 19, 18), (326, 'no', 'yes', 11, 3895, 96, 1, 5, 28, 1), (347, 'yes', 'no', 11, 3895, 107, 9, 16, 10, 9), (692, 'yes', 'no', 11, 3895, 116, 29, 25, 25, 24), (868, 'yes', 'no', 11, 3895, 112, 23, 21, 21, 19), (1347, 'yes', 'no', 11, 3895, 104, 8, 13, 13, 11), (2760, 'yes', 'no', 11, 3895, 129, 23, 38, 38, 35), (3354, 'yes', 'no', 11, 3895, 123, 26, 32, 32, 30), (4742, 'yes', 'no', 2, 3895, 151, 45, 60, 60, 59)]



[(0, 259, datetime.date(2020, 3, 25), '50s', 'MALE', 'NO KNOWN EPI LINK', 'No', 'Toronto'), (1, 5981, datetime.date(2020, 3, 16), '20s', 'MALE', 'TRAVEL', 'No', 'Toronto'), (2, 7345, datetime.date(2020, 3, 23), '50s', 'MALE', 'OB', 'Yes', 'Toronto'), (3, 7354, datetime.date(2020, 3, 24), '20s', 'FEMALE', 'CC', 'No', 'Toronto'), (4, 8096, datetime.date(2020, 3, 28), '70s', 'FEMALE', 'TRAVEL', 'No', 'Ottawa'), (5, 8319

In [3]:
individual_df = pd.DataFrame(individual_table, columns=['surrogate_key', 'individual_id', 'date', 'age_group', 'gender', 'cause_of_infection', 'outbreak_related', 'reporting_phu_city'])
individual_df.head()

,surrogate_key,individual_id,date,age_group,gender,cause_of_infection,outbreak_related,reporting_phu_city
0,0,259,2020-03-25,50s,MALE,NO KNOWN EPI LINK,No,Toronto
1,1,5981,2020-03-16,20s,MALE,TRAVEL,No,Toronto
2,2,7345,2020-03-23,50s,MALE,OB,Yes,Toronto
3,3,7354,2020-03-24,20s,FEMALE,CC,No,Toronto
4,4,8096,2020-03-28,70s,FEMALE,TRAVEL,No,Ottawa


### Label Encoding for Cause of Infection

In [4]:
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
individual_df['cause_of_infection_Cat'] = labelencoder.fit_transform(individual_df['cause_of_infection'])
individual_df.head()

,surrogate_key,individual_id,date,age_group,gender,cause_of_infection,outbreak_related,reporting_phu_city,cause_of_infection_Cat
0,0,259,2020-03-25,50s,MALE,NO KNOWN EPI LINK,No,Toronto,2
1,1,5981,2020-03-16,20s,MALE,TRAVEL,No,Toronto,4
2,2,7345,2020-03-23,50s,MALE,OB,Yes,Toronto,3
3,3,7354,2020-03-24,20s,FEMALE,CC,No,Toronto,0
4,4,8096,2020-03-28,70s,FEMALE,TRAVEL,No,Ottawa,4


In [5]:
# creating instance of one-hot-encoder

enc = OneHotEncoder(handle_unknown='ignore')

In [6]:
# passing bridge-types-cat column (label encoded values of bridge_types)

enc_df = pd.DataFrame(enc.fit_transform(individual_df[['cause_of_infection_Cat']]).toarray())


In [7]:
# merge with main df bridge_df on key values

individual_df = individual_df.join(enc_df)
individual_df.rename(columns={0: "ac_cc", 1: "ac_", 2: "ac_nkel", 3: "ac_ob", 4: "ac_travel", 5: "ac_mi"},inplace = True)
individual_df=individual_df.drop(['surrogate_key', 'individual_id', 'date', 'age_group', 'gender', 'cause_of_infection', 'outbreak_related', 'reporting_phu_city','cause_of_infection_Cat'], axis=1)
individual_df.head()

,ac_cc,ac_,ac_nkel,ac_ob,ac_travel,ac_mi
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0
